In [4]:
#!pip install tensorflow

In [5]:
import pandas as pd
import numpy as np
import json
from tensorflow import keras
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


In [6]:
x_src_path = '../Dataset/assignment1New.json'
y_src_path = '../DataBook/Assignment1_Data_Analyst.xlsx'

In [7]:
data = pd.read_json(x_src_path)


In [8]:
df_supervision = pd.read_excel(y_src_path)
plagiarised_array = df_supervision['Plagiarised'].astype(int).values

In [9]:
# X = np.array(X_flattened_padded)
X = data.values
y = plagiarised_array

In [10]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_imputed = imputer.fit_transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

In [12]:
# let's train a perceptron on these images

# first, we need to flatten our images (from 2d into a 1d array)
x_train_flat = X_train.reshape(X_train.shape[0], -1)
x_test_flat = X_test.reshape(X_test.shape[0], -1)

# create and train the perceptron
perceptron = Perceptron()
perceptron.fit(x_train_flat, y_train)

# print train and test accuracy scores
print(f"Training data score: {perceptron.score(x_train_flat, y_train)}")
print(f"Training data score: {perceptron.score(x_test_flat, y_test)}")


Training data score: 0.5116279069767442
Training data score: 0.45454545454545453


In [13]:
# let's use tensorflow keras library to build a 10 perceptron model
#  Dense layers are standard fully-interconnected neural network layers
model = keras.models.Sequential([
    keras.layers.Dense(10, activation='softmax') # output layer
])

# we'll use sparse_categorical_crossentropy as the loss function
LOSS_FN = keras.losses.sparse_categorical_crossentropy

# compile the model with standard backprop training algorithm called 'adam'
model.compile(optimizer='adam',loss=LOSS_FN,metrics=['accuracy'])

# train on training data, and validate on test data
# we'll train for 5 epochs
model.fit(x_train_flat, y_train, epochs=5, validation_data=(x_test_flat, y_test))



Epoch 1/5


2/2 [==============================] - 1s 157ms/step - loss: 18735.5586 - accuracy: 0.5581 - val_loss: 109528.8828 - val_accuracy: 0.4545
Epoch 2/5
2/2 [==============================] - 0s 31ms/step - loss: 90656.9844 - accuracy: 0.5116 - val_loss: 15495.3848 - val_accuracy: 0.4545
Epoch 3/5
2/2 [==============================] - 0s 29ms/step - loss: 12758.3867 - accuracy: 0.6744 - val_loss: 84932.7266 - val_accuracy: 0.5455
Epoch 4/5
2/2 [==============================] - 0s 28ms/step - loss: 97837.4062 - accuracy: 0.4884 - val_loss: 93305.0781 - val_accuracy: 0.5455
Epoch 5/5
2/2 [==============================] - 0s 29ms/step - loss: 99092.5078 - accuracy: 0.4884 - val_loss: 45947.2656 - val_accuracy: 0.5455


In [14]:
# let's build a whole bunch of models to compare
modelNames = [
    '1d',
    '2d',
    '3d',
    '1c1d',
    '1c2d',
    '2c2d',
]

# in all models we'll have the same number of nodes per hidden layer
NODES_PER_HIDDEN_LAYER = 15

models = [
    # just the output layer
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 1 hidden layer
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 2 hidden layers
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),   # hidden layer 2
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 1 convolutional layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")  # output layer
    ]),
    # 1 convolutional layer and 1 hidden dense layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Flatten(),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation="softmax")  # output layer
    ]),
    # 2 convolutional layers and 1 hidden dense layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 2
        keras.layers.Flatten(),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation="softmax")  # output layer
    ])
]



In [15]:
# compile all models
for model in models:
    model.compile(optimizer='adam',loss=LOSS_FN,metrics=['accuracy'])

In [16]:
TRAINING_EPOCHS = 5

# train all models
for model, name in zip(models, modelNames):
    print(f'training model {name}')
    model.fit(X_train, y_train, epochs=TRAINING_EPOCHS)


training model 1d
Epoch 1/5


ValueError: in user code:

    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\K-Gen\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File

In [ ]:
# get all model accuracy scores on test data
scores = [model.evaluate(X_test,y_test)[1] for model in models]


In [ ]:
# let's also add the perceptron score to our accuracy scores list
modelNames.insert(0, 'perceptron')
scores.insert(0, perceptron.score(x_test_flat, y_test) )

# print all models and scores
for name,score in zip(modelNames,scores):
    print(name, score)

In [ ]:
# display as a bar chart
plt.bar(modelNames,scores)
plt.ylim(0.8, 1.0)
plt.show()